# Advanced Econometrics (JEM217) – Empirical Project (WS 2025)

- **Title:      Determinants of Lap Times in Formula 1**
- **Authors:    Sebastian Pasz \& Vojtěch Dohnal**

**AI Usage**

ChatGPT-5 was used during the preparation of this assignment **solely for grammatical editing and minor code optimization**.  
Any sections where AI-assisted code optimization was applied are explicitly indicated within the notebook.

## Initial Preparation

In this section, we install and load all required R packages necessary for data simulation, estimation, and visualization.

In [2]:
req_pkgs <- c("ggplot2","dplyr","quantreg","lmtest","sandwich","car","stargazer","knitr","glm2","gmm","msm","plm","modelsummary","readxl","lubridate","stringr")
for (p in req_pkgs) {
  if (!requireNamespace(p, quietly = TRUE)) {
    install.packages(p, repos = "https://cloud.r-project.org")
  }
}
library(ggplot2)
library(dplyr)
library(quantreg)
library(lmtest)
library(sandwich)
library(car)
library(stargazer)
library(knitr)
library(glm2)
library(gmm)
library(msm)
library(plm)
library(modelsummary)
library(readxl)
library(lubridate)
library(stringr)

Warning message:
"package 'stringr' was built under R version 4.4.3"


In [4]:
data <- read_excel("./Dataset_2025.xlsx")

print(sum(!complete.cases(data[, c("Driver", "LapTime", "LapNumber", "Compound", "TyreLife", 
                             "Team", "TrackStatus", "Position", "Deleted", "AirTemp", 
                             "Humidity", "Pressure", "Rainfall", "TrackTemp", "WindSpeed")])))

str(data)

[1] 29145
tibble [29,145 × 41] (S3: tbl_df/tbl/data.frame)
 $ year              : num [1:29145] 2025 2025 2025 2025 2025 ...
 $ country           : chr [1:29145] "Abu Dhabi" "Abu Dhabi" "Abu Dhabi" "Abu Dhabi" ...
 $ session           : chr [1:29145] "Race" "Race" "Race" "Race" ...
 $ Time              : num [1:29145] 0.0416 0.0416 0.0416 0.0416 0.0416 ...
 $ Driver            : chr [1:29145] "VER" "PIA" "NOR" "LEC" ...
 $ DriverNumber      : chr [1:29145] "1" "81" "4" "16" ...
 $ LapTime           : num [1:29145] 0.00106 0.00108 0.00109 0.0011 0.0011 ...
 $ LapNumber         : num [1:29145] 1 1 1 1 1 1 1 1 1 1 ...
 $ Stint             : num [1:29145] 1 1 1 1 1 1 1 1 1 1 ...
 $ PitOutTime        : num [1:29145] NA NA NA NA NA NA NA NA NA NA ...
 $ PitInTime         : num [1:29145] NA NA NA NA NA NA NA NA NA NA ...
 $ Sector1Time       : num [1:29145] NA NA NA NA NA NA NA NA NA NA ...
 $ Sector2Time       : num [1:29145] 0.000445 0.000444 0.000448 0.00045 0.000447 ...
 $ Sector3Time    